In [1]:
import numpy as np
import pandas as pd
import PIL
import keras
import itertools
from PIL import Image
# import ibm_boto3
# from botocore.client import Config


from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from skimage import feature, data, io, measure
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

from keras import backend as K
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import sys
import zipfile
import types

Using TensorFlow backend.


In [2]:
zipdata = zipfile.ZipFile("Data.zip")

In [3]:
paths = zipdata.namelist()
print(paths)

['Data/', 'Data/debug.log', 'Data/Test_Data/', 'Data/Test_Data/Aadhar/', 'Data/Test_Data/Aadhar/00000003.jpg', 'Data/Test_Data/Insurance/', 'Data/Test_Data/Insurance/00000017.jpg', 'Data/Test_Data/Receipts/', 'Data/Test_Data/Receipts/00000032.jpg', 'Data/Train_Data/', 'Data/Train_Data/Aadhar/', 'Data/Train_Data/Aadhar/00000000.jpg', 'Data/Train_Data/Aadhar/00000001.jpg', 'Data/Train_Data/Aadhar/00000002.jpg', 'Data/Train_Data/Aadhar/00000004.jpg', 'Data/Train_Data/Aadhar/00000005.jpg', 'Data/Train_Data/Aadhar/00000006.jpg', 'Data/Train_Data/Aadhar/00000007.jpg', 'Data/Train_Data/Aadhar/00000008.jpg', 'Data/Train_Data/Aadhar/00000009.jpg', 'Data/Train_Data/Aadhar/00000010.jpg', 'Data/Train_Data/Aadhar/00000011.jpg', 'Data/Train_Data/Aadhar/00000012.jpg', 'Data/Train_Data/Aadhar/00000013.jpg', 'Data/Train_Data/Aadhar/00000014.jpg', 'Data/Train_Data/Aadhar/00000015.jpg', 'Data/Train_Data/Aadhar/00000016.jpg', 'Data/Train_Data/Aadhar/00000017.jpg', 'Data/Train_Data/Aadhar/00000018.jpg', 'D

In [4]:
batch_size_train = 264
batch_size_val = 10
batch_size_test = 25
num_classes= 3
intereseted_folder='Aadhar'
STANDARD_SIZE=(224,224)

In [5]:
classes_required=[]
for path in paths:
    zipdata.extract(path)
    temp=path.split('/')
    if len(temp) > 3:
        if temp[2] not in classes_required:
            classes_required.append(temp[2])
print(classes_required)
zipdata.close()

['Aadhar', 'Insurance', 'Receipts']


In [6]:
datagen=keras.preprocessing.image.ImageDataGenerator(data_format=K.image_data_format())

In [7]:
train_path = 'Data\Train_Data'
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224), classes=classes_required, batch_size=batch_size_train)
type(train_batches)

Found 72 images belonging to 3 classes.


keras.preprocessing.image.DirectoryIterator

In [8]:
val_path = 'Data/Val_Data/'
val_batches = ImageDataGenerator().flow_from_directory(val_path, target_size=(224,224), classes=classes_required, batch_size=batch_size_val)

Found 3 images belonging to 3 classes.


In [9]:
test_path = 'Data/Test_Data/'
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), classes=classes_required, batch_size=batch_size_test)

Found 3 images belonging to 3 classes.


In [10]:
test_imgs, test_labels = next(test_batches)
test_labels

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)

In [11]:
y_test= [ np.where(r==1)[0][0] for r in test_labels ]
y_test

[0, 2, 1]

In [12]:
vgg16_model = keras.applications.vgg16.VGG16()
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [13]:
type(vgg16_model) #This is a Keras Functional API need to convert to sequential
model = Sequential() #Iterate over the functional layers and add it as a stack
for layer in vgg16_model.layers:
    model.add(layer)

In [14]:
model.layers.pop()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [15]:
for layer in model.layers: #Since the model is already trained with certain weights, we dont want to change it. Let it be the same
    layer.trainable = False

In [16]:
model.add(Dense(3, activation='sigmoid')) # Add the last layer
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [17]:
# Complie the model
model.compile(Adam(lr=.00015), loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.fit_generator(train_batches, steps_per_epoch = 18,
                    validation_data=val_batches, epochs = 15)

Epoch 1/15


d:\softwares\python\lib\site-packages\PIL\Image.py:989: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


18/18 [==============================] - 35s 2s/step - loss: 1.0934 - accuracy: 0.4483 - val_loss: 1.0900 - val_accuracy: 0.6667
Epoch 2/15
18/18 [==============================] - 22s 1s/step - loss: 1.0925 - accuracy: 0.4823 - val_loss: 1.0895 - val_accuracy: 0.6667
Epoch 3/15
18/18 [==============================] - 22s 1s/step - loss: 1.0916 - accuracy: 0.5231 - val_loss: 1.0890 - val_accuracy: 0.6667
Epoch 4/15
18/18 [==============================] - 22s 1s/step - loss: 1.0907 - accuracy: 0.5687 - val_loss: 1.0884 - val_accuracy: 0.6667
Epoch 5/15
18/18 [==============================] - 22s 1s/step - loss: 1.0898 - accuracy: 0.6142 - val_loss: 1.0879 - val_accuracy: 0.6667
Epoch 6/15
18/18 [==============================] - 22s 1s/step - loss: 1.0890 - accuracy: 0.6366 - val_loss: 1.0874 - val_accuracy: 0.6667
Epoch 7/15
18/18 [==============================] - 22s 1s/step - loss: 1.0881 - accuracy: 0.6451 - val_loss: 1.0868 - val_accuracy: 0.6667
Epoch 8/15
18/18 [=============

In [19]:
zip = zipfile.ZipFile("test_doc-external.zip")
paths = zip.namelist()
del paths[0]
print(paths)
for path in paths:
    print(zip.extract(path))
zip.close()

['test_doc-external/00000046.jpg', 'test_doc-external/00000197.jpg', 'test_doc-external/aadhar copy 2.jpg']
D:\College\BE Project\test_doc-external\00000046.jpg
D:\College\BE Project\test_doc-external\00000197.jpg
D:\College\BE Project\test_doc-external\aadhar copy 2.jpg


In [20]:
X_test=[]
def convert_to_image(X):
    '''Function to convert all Input Images to the STANDARD_SIZE and create Training Dataset
    '''
    for f in paths:
        #fobj=get_file(f)
        #print(type(fobj))predictions= model.predict(X_test)
        if os.path.isdir(f):
            continue
        img= PIL.Image.open(f)
        
        img = img.resize(STANDARD_SIZE)
        print(img)
        img=np.array(img)
        X.append(img)
        #print(X_train)
    #print(len(X_train))
    return X
X_test=np.array(convert_to_image(X_test))
datagen.fit(X_test)

<PIL.Image.Image image mode=RGB size=224x224 at 0x19212403898>
<PIL.Image.Image image mode=RGB size=224x224 at 0x19212403898>
<PIL.Image.Image image mode=RGB size=224x224 at 0x19212403898>


In [21]:
predictions= model.predict(X_test)
predictions

array([[0.50948554, 0.49298924, 0.46034196],
       [0.4799884 , 0.50090444, 0.48431385],
       [0.5105191 , 0.49314758, 0.4586537 ]], dtype=float32)

In [25]:
y_pred=[]
for i in range(len(predictions)):
    y_pred.append(np.argmax(predictions[i]))
y_pred
# j = 0
# for i in y_pred:
#     print(paths[y_pred[j]-1])
#     j = j + 1

[0, 1, 0]

In [29]:
index= classes_required.index('Aadhar')
for i in range(len(y_pred)):
    if y_pred[i] == index:
        print("Image classified as a form document: ", paths[i])

Image classified as a form document:  test_doc-external/00000046.jpg
Image classified as a form document:  test_doc-external/aadhar copy 2.jpg


In [30]:
index= classes_required.index('Insurance')
for i in range(len(y_pred)):
    if y_pred[i] == index:
        print("Image classified as a form document: ", paths[i])

Image classified as a form document:  test_doc-external/00000197.jpg
